A idéia deste notebook é explorar diversos métodos de avaliação de impacto de variáveis com diferentes tipos de modelos, técnicas e métricas.

# Imports

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings

from pathlib import Path
from tqdm.notebook import tqdm
from collections import defaultdict
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import RobustScaler

# Configs

In [2]:
data_path = Path(r'..\..\01_dados\01_dados_analise\01_dados_anomalia.csv')

models = [ 
    SVR(kernel="linear"), 
    GradientBoostingRegressor(),
    DecisionTreeRegressor(),
    XGBRegressor(objective ='reg:squarederror'), 
    RandomForestRegressor(),
    LinearRegression()
]

ranks = [30,57,11,42, 29, 54, 56, 55, 19, 3, 12, 15, 1, 43, 8, 
        2, 23, 24, 33, 28, 61, 31, 41, 45, 4, 20, 14, 13, 5, 10, 
        18, 0, 21, 25, 27, 16, 26, 6, 39, 7, 52, 22, 53,44, 40, 46, 
        34, 38, 32, 51, 9, 17, 62, 50, 35, 47, 37, 36, 49, 48, 58, 59, 60]

# Funções e Classes de objetos

In [3]:
class MultiModelRFE:
    """
    Classe para seleção recursiva de características usando múltiplos modelos.

    Esta classe aplica o método RFE (Recursive Feature Elimination) usando diferentes modelos para
    determinar a importância das características. Posteriormente, as características são classificadas
    com base na sua frequência de seleção.

    Atributos:
    -----------
    models: list
        Lista de modelos de regressão.
    feature_counts: dict
        Dicionário para contar quantas vezes cada característica foi selecionada.
    sorted_features: list
        Lista de características classificadas por frequência de seleção.
    """

    def __init__(self, models):
        """
        Inicializa a classe com os modelos fornecidos.

        Parâmetros:
        ----------
        models: list
            Lista de modelos de regressão.
        """
        self.models = models
        self.feature_counts = defaultdict(int)

    def fit(self, X, y, n_iterations=50, sample_ratio=0.005):
        """
        Ajusta os modelos ao conjunto de dados para determinar a importância das características.
        Usa amostragem aleatória para acelerar o processo.

        Parâmetros:
        ----------
        X: array-like, shape (n_samples, n_features)
            Conjunto de dados com amostras e características.
        y: array-like, shape (n_samples,)
            Valores-alvo.
        n_iterations: int, opcional (padrão=100)
            Número de iterações para RFE.
        sample_ratio: float, opcional (padrão=0.1)
            Proporção de dados a serem amostrados em cada iteração.

        Retorna:
        -------
        list
            Lista de características classificadas pela sua frequência de seleção.
        """
        n_samples = int(sample_ratio * X.shape[0])

        for model in tqdm(self.models, desc="Processando Modelos", unit="modelo"):
            with tqdm(total=n_iterations, desc="Extração de Características RFE", leave=False, unit="iteração") as pbar:
                for _ in range(n_iterations):
                    # Amostragem aleatória dos dados
                    idx = np.random.choice(X.shape[0], n_samples, replace=False)
                    X_sample = X[idx]
                    y_sample = y[idx]

                    selector = RFE(model, n_features_to_select=1, step=1)
                    selector.fit(X_sample, y_sample)

                    # Atualiza contagem das características baseado nos rankings do seletor
                    for feature_idx, rank in enumerate(selector.ranking_):
                        self.feature_counts[feature_idx] += (X.shape[1] - rank)

                    pbar.update(1)

        self.sorted_features = sorted(self.feature_counts.keys(), key=lambda x: self.feature_counts[x], reverse=True)
        return self.sorted_features

    def get_ranked_features(self):
        """
        Retorna as características classificadas pela sua frequência de seleção.

        Retorna:
        -------
        list
            Lista de características classificadas.
        """
        return self.sorted_features

# Importação e preparação dos dados

In [4]:
df = pd.read_csv(data_path)
df.head()

,id_subsistema,din_instante,"Geração no Centro de Gravidade - MW médios (Gp,j) - MWh","Fator de Abatimento de Perdas Internas Instantâneas (F_PDIp,j)","Fator de Rateio das Perdas de Geração (UXP_GLFp,j)*","Deslocamento Hidráulico Energético Preliminar (DH_ENER_PRE_UHp,j)","Garantia física modulada ajustada pelo fator de disponibilidade (GFIS_2p,j)","Garantia Física de Repasse de Risco Hidrológico Modulada e Ajustada (GFIS_2_RRHp,j) - MWh","Garantis Física Modulada Ajustada de Repasse do Risco Hidrológico (GFIS_3_RRHp,j) - MWh","Fator de Risco Hidrológico aceito pelo gerador, variando entre 0 e 11% (Fp,j)",...,val_verifconstrainedoff,val_importacaoprogramada,val_importacaodespachada,val_importacaoverificada,val_preco_importacao,val_dispf,val_indisppf,val_indispff,val_intercambiomwmed,anomaly
0,N,2018-01-06 00:00:00,8564.245664,18.764152,18.688987,108.091180,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,S,2018-01-06 00:00:00,7715.282975,213.306544,213.352056,89.507131,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,NE,2018-01-06 00:00:00,2825.618028,19.924679,19.714905,68.291389,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,SE,2018-01-06 00:00:00,30843.187134,373.242309,374.237593,371.119404,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,SE,2018-01-06 01:00:00,30513.088050,372.459187,374.129311,327.623405,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [5]:
# colocando o cmo no final do dataframe para facilitar a manipulação dos dados
cols = [col for col in df if col != 'cmo'] + ['cmo']
df = df[cols]

In [6]:
df_aux = df.copy()
df_aux.drop(['id_subsistema', 'din_instante'], inplace=True, axis=1)

In [7]:
# Separando as variáveis para modelagem X são as variáveis independentes e Y a Variável indepentende
X = df_aux.iloc[:, :-1].values 
Y = df_aux.iloc[:, -1].values 

scaler = RobustScaler()

X = scaler.fit_transform(X)

# Método 1: Recursive Feature Elimination (RFE)

A *Eliminação Recursiva de Características (RFE)* é uma técnica utilizada para selecionar as características mais importantes de um conjunto de dados. O princípio subjacente da RFE é recursivamente remover características e construir um modelo com as características restantes, avaliando a performance do modelo para determinar a importância de cada característica. As características são, então, ranqueadas baseadas na ordem em que foram eliminadas.

## Como funciona?

- **Primeiro:** Um modelo é treinado no conjunto de dados original.
- **Segundo:** A importância de cada característica é obtida. Isso pode ser através de coeficientes para modelos lineares ou `feature_importances_` para modelos baseados em árvores.
- **Terceiro:** A característica menos importante é removida do conjunto de dados.
- **Quarto:** O modelo é treinado novamente usando o conjunto de dados reduzido.
- **Quinto:** O processo é repetido até que um número especificado de características seja atingido ou até que todas as características sejam eliminadas.

## Sobre a classe `MultiModelRFE`:

A classe `MultiModelRFE` estende a ideia básica de RFE para usar múltiplos modelos. Isso pode ser útil porque diferentes modelos podem ter diferentes opiniões sobre a importância das características, e ao usar vários modelos, você pode obter um consenso mais robusto sobre quais características são verdadeiramente importantes.

## Funcionalidades:

- `__init__(self, models)`: O construtor aceita uma lista de modelos, que devem ser modelos de regressão que implementam `coef_` ou `feature_importances_`.
  
- `fit(self, X, y)`: Este método ajusta todos os modelos fornecidos ao conjunto de dados. Determina a importância das características e consolida os rankings através de todos os modelos.
  
- `get_ranked_features()`: Retorna as características ranqueadas pela sua frequência de aparição através de todos os modelos.

## Considerações:

- Esta classe é útil se você está indeciso sobre qual modelo usar e gostaria de ter uma visão geral de várias perspectivas.
- A acumulação de importâncias através de múltiplos modelos oferece uma visão robusta da real importância das características.
- O RFE pode ser intensivo em termos computacionais. Considere usar métodos mais rápidos ou um subconjunto dos dados em conjuntos de dados grandes.
- Importe as bibliotecas e classes necessárias, como `RFE` e `defaultdict`.

In [8]:
multi_rfe = MultiModelRFE(models)
ranked_features = multi_rfe.fit(X, Y)

Processando Modelos:   0%|          | 0/6 [00:00<?, ?modelo/s]

Extração de Características RFE:   0%|          | 0/50 [00:00<?, ?iteração/s]

Extração de Características RFE:   0%|          | 0/50 [00:00<?, ?iteração/s]

Extração de Características RFE:   0%|          | 0/50 [00:00<?, ?iteração/s]

Extração de Características RFE:   0%|          | 0/50 [00:00<?, ?iteração/s]

Extração de Características RFE:   0%|          | 0/50 [00:00<?, ?iteração/s]

Extração de Características RFE:   0%|          | 0/50 [00:00<?, ?iteração/s]

[30,57,11,42,
 29,
 54,
 56,
 55,
 19,
 3,
 12,
 15,
 1,
 43,
 8,
 2,
 23,
 24,
 33,
 28,
 61,
 31,
 41,
 45,
 4,
 20,
 14,
 13,
 5,
 10,
 18,
 0,
 21,
 25,
 27,
 16,
 26,
 6,
 39,
 7,
 52,
 22,
 53,
 44,
 40,
 46,
 34,
 38,
 32,
 51,
 9,
 17,
 62,
 50,
 35,
 47,
 37,
 36,
 49,
 48,
 58,
 59,
 60]

In [17]:
feature_rank = [i-1 for i in ranks]

features_names = df_aux.columns.to_list()
features_names.pop()

ranked_features = [name for idx, name in sorted(zip(feature_rank, features_names))]

In [18]:
ranked_features

['val_proginflexibilidade',
 'val_gereolica',
 'val_intercambio',
 'Garantia física modulada ajustada pelo fator de disponibilidade (GFIS_RBp,j * F_DISPp,m)',
 'val_folgadegeracao',
 'val_proggeracao',
 'val_progreservapotencia',
 'val_progunitcommitment',
 'val_carga',
 'val_atendsatisfatoriorpo',
 'val_progordemmerito',
 'Fator de Rateio das Perdas de Geração (UXP_GLFp,j)*',
 'val_gerhidraulica',
 'val_energiavertidaturbinavel',
 'val_vazaovertidaturbinavel',
 'val_gertermica',
 'val_progreposicaoperdas',
 'val_verifgsub',
 'val_progordemdemeritoref',
 'Garantias Física Modulada Ajustada de Repasse do Risco Hidrológico (GFIS_3_RRHp,j) - MWh',
 'val_energiavertida',
 'val_prograzaoeletrica',
 'val_verifgeracao',
 'val_cargaenergiahomwmed',
 'num_ordemcs',
 'val_proggarantiaenergetica',
 'val_progexportacao',
 'val_proggfom',
 'val_disponibilidade',
 'Garantia física modulada ajustada pelo fator de disponibilidade (GFIS_2p,j)',
 'Geração no Centro de Gravidade - MW médios (Gp,j) - MWh'

Resultado do Método 1, agora desenvolver melhores análises para validar esse resultado